 # WebSecure: Système Avancé d'Analyse de Vulnérabilités Web
    
  **Auteurs: ELGARCH Youssef et IBNOU-KADY Nisrine**
    
  ## Introduction

  Ce notebook présente le système WebSecure, une plateforme d'analyse de vulnérabilités web qui combine des techniques d'apprentissage automatique et d'analyse de sites web pour détecter les risques de sécurité potentiels.
  
  Le système est capable de :
    - Analyser les sites web pour détecter les vulnérabilités
    - Classifier les niveaux de risque (Faible, Modéré, Important, Critique),
    - Détecter les anomalies de sécurité,
    - Générer des rapports détaillés avec recommandations,
    - S'intégrer avec une application web pour une utilisation simplifiée,
  
  ### Objectifs du projet
  
    1. Développer un modèle de machine learning pour prédire les risques de sécurité
    2. Créer un système d'extraction automatique des caractéristiques de sécurité d'un site web
    3. Concevoir une interface utilisateur intuitive pour l'analyse de sécurité
    4. Produire des rapports pertinents et actionnables pour les utilisateurs
    
 ### Prérequis et dépendances
    
    Le notebook utilise plusieurs bibliothèques Python pour l'analyse et l'apprentissage automatique :


In [ ]:
!pip install -U scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import joblib
import warnings
import time
from datetime import datetime
from tqdm import tqdm
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, roc_auc_score, precision_recall_curve
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import warnings
warnings.filterwarnings("ignore")

# Configuration des bibliothèques NLTK
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#SECTION 1: PRÉPARATION ET PRÉTRAITEMENT DES DONNÉES

In [ ]:
class DataPreprocessor:
    """Classe pour le prétraitement des données de vulnérabilité"""

    def __init__(self, config=None):
        self.config = config or {
            'text_cleaning': True,
            'handle_missing': 'fill_empty',  # 'drop', 'fill_empty', 'fill_median'
            'scaling_method': 'standard',    # 'standard', 'minmax', 'robust'
            'feature_selection': True,
            'n_features': 20,
            'stemming': True,
            'language': 'french',
            'apply_smote': False
        }
        # Explicitly set stemming as an attribute
        self.stemming = self.config['stemming']
        self.scaler = None
        self.feature_selector = None

        # Téléchargement des données NLTK si nécessaire
        try:
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            nltk.download('punkt')

        try:
            nltk.data.find('corpora/stopwords')
        except LookupError:
            nltk.download('stopwords')

        self.stemmer = SnowballStemmer(self.config['language']) if self.stemming else None
        self.stop_words = list(stopwords.words(self.config['language'])) if self.stemming else None
        self.stemmer = SnowballStemmer(self.config['language']) if self.stemming else None
        self.stop_words = list(stopwords.words(self.config['language'])) if self.stemming else None


    def load_and_preprocess(self, file_path):
        """Charge et prétraite les données depuis un fichier CSV"""
        print(f"\n📂 Chargement des données depuis: {file_path}...")
        try:
            df = pd.read_csv(file_path)
            print(f"   ✓ Données chargées: {df.shape[0]} entrées, {df.shape[1]} colonnes")
        except FileNotFoundError:
            raise FileNotFoundError(f"⚠️ Fichier non trouvé: {file_path}")
        except Exception as e:
            raise Exception(f"⚠️ Erreur lors du chargement du fichier: {e}")



        # Prétraitement
        print("   - Prétraitement des données...")
        df = self.handle_missing_values(df)

        # Normalisation des niveaux de risque et d'impact (added code)
        risk_level_map = {
            'Critique': 3,
            'Important': 2,
            'Modéré': 1,
            'No Risk Level': 0  # Include mapping for 'No Risk Level' if present
        }

        impact_level_map = {
            'Critique': 3,
            'Important': 2,
            'Modéré': 1,
            'No Impact Level': 0  # Include mapping for 'No Impact Level' if present
        }

        df['Risk Level Numeric'] = df['Risk Level'].map(risk_level_map)
        df['Impact Level Numeric'] = df['Impact Level'].map(impact_level_map)

        # Combiner les colonnes de texte pour l'extraction des caractéristiques
        df['Combined_Text'] = df['Vulnerability Summary'] + ' ' + df['Affected Systems'] + ' ' + df['Solution']

        # Convertir en minuscules et supprimer les caractères spéciaux (facultatif)
        if self.config['text_cleaning']:
            df['Combined_Text'] = df['Combined_Text'].apply(self.clean_text)

        print("   ✓ Données prétraitées.")
        return df




    def handle_missing_values(self, df):
        """Gère les valeurs manquantes dans le DataFrame"""
        if self.config['handle_missing'] == 'drop':
           df = df.dropna()
        elif self.config['handle_missing'] == 'fill_empty':
        # Fill missing values with empty strings for string columns only
             for col in df.select_dtypes(include=['object']).columns:
                 df[col] = df[col].fillna('')
        # Fill missing values with 0 for numeric columns only
             for col in df.select_dtypes(include=np.number).columns:
                 df[col] = df[col].fillna(0)
        elif self.config['handle_missing'] == 'fill_median':
             for col in df.select_dtypes(include=np.number).columns:
                 df[col] = df[col].fillna(df[col].median())
        return df




    def clean_text(self, text):
        """Nettoie le texte en supprimant les caractères spéciaux et en convertissant en minuscules"""
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)  # Supprime les caractères spéciaux
        text = text.strip()
        if self.stemming:  # Applique le stemming si activé
            tokens = nltk.word_tokenize(text)
            stemmed_tokens = [self.stemmer.stem(token) for token in tokens if token not in self.stop_words]
            text = ' '.join(stemmed_tokens)
        return text

    def extract_features(self, df):
        """Extrait des caractéristiques pour l'analyse de vulnérabilité"""
        print("\n🔧 Extraction des caractéristiques...")

         # Vérification des NaN dans Risk Level Numeric avant tout traitement
        if df['Risk Level Numeric'].isna().any():
            print("   ⚠️ Valeurs manquantes détectées dans Risk Level Numeric. Remplacement par 0...")
            df['Risk Level Numeric'] = df['Risk Level Numeric'].fillna(0)

    # Caractéristiques textuelles
        if 'Combined_Text' in df.columns:
           text_column = 'Combined_Text'
        else:
           text_column = 'Vulnerability Summary'

        print(f"   - Utilisation de la colonne '{text_column}' pour l'extraction textuelle")

    # Vectorisation des descriptions
        tfidf = TfidfVectorizer(max_features=1000, stop_words=self.stop_words)
        X_text = tfidf.fit_transform(df[text_column])
        print(f"   ✓ Caractéristiques textuelles extraites: {X_text.shape[1]} dimensions")

    # Enregistrement du vectoriseur
        joblib.dump(tfidf, 'models/tfidf_vectorizer.pkl')

    # Extraction de caractéristiques basées sur les règles
        features = []
        print("   - Extraction des caractéristiques basées sur les règles...")

        for idx, row in tqdm(df.iterrows(), total=df.shape[0], desc="   🔄 Traitement des entrées"):
           affected_systems = str(row['Affected Systems']).lower()
           vuln_summary = str(row['Vulnerability Summary']).lower()

           feature_dict = {
                # Systèmes affectés
                'has_windows': 1 if 'windows' in affected_systems else 0,
                'has_linux': 1 if 'linux' in affected_systems else 0,
                'has_macos': 1 if ('macos' in affected_systems or 'mac os' in affected_systems) else 0,
                'has_android': 1 if 'android' in affected_systems else 0,
                'has_ios': 1 if ('ios' in affected_systems or 'iphone' in affected_systems or 'ipad' in affected_systems) else 0,
                'has_wordpress': 1 if 'wordpress' in affected_systems else 0,
                'has_joomla': 1 if 'joomla' in affected_systems else 0,
                'has_drupal': 1 if 'drupal' in affected_systems else 0,
                'has_oracle': 1 if 'oracle' in affected_systems else 0,
                'has_microsoft': 1 if 'microsoft' in affected_systems else 0,
                'has_adobe': 1 if 'adobe' in affected_systems else 0,
                'has_plugin': 1 if 'plugin' in affected_systems or 'extension' in affected_systems else 0,
                'has_server': 1 if 'server' in affected_systems or 'serveur' in affected_systems else 0,
                'has_web': 1 if 'web' in affected_systems else 0,
                'has_mobile': 1 if 'mobile' in affected_systems else 0,
                'has_iot': 1 if 'iot' in affected_systems or 'internet of things' in affected_systems else 0,
                'has_cloud': 1 if 'cloud' in affected_systems or 'aws' in affected_systems or 'azure' in affected_systems else 0,

                # Types de vulnérabilités
                'has_injection': 1 if 'injection' in vuln_summary else 0,
                'has_xss': 1 if 'xss' in vuln_summary or 'cross-site' in vuln_summary or 'script' in vuln_summary else 0,
                'has_sql': 1 if 'sql' in vuln_summary else 0,
                'has_authentication': 1 if 'authentification' in vuln_summary or 'authentication' in vuln_summary else 0,
                'has_authorization': 1 if 'autorisation' in vuln_summary or 'authorization' in vuln_summary else 0,
                'has_privilege': 1 if 'privilège' in vuln_summary or 'privilege' in vuln_summary else 0,
                'has_code_execution': 1 if 'exécution de code' in vuln_summary or 'code execution' in vuln_summary else 0,
                'has_arbitrary_code': 1 if 'arbitraire' in vuln_summary or 'arbitrary' in vuln_summary else 0,
                'has_buffer': 1 if 'buffer' in vuln_summary or 'tampon' in vuln_summary else 0,
                'has_overflow': 1 if 'overflow' in vuln_summary or 'débordement' in vuln_summary else 0,
                'has_dos': 1 if 'déni de service' in vuln_summary or 'denial of service' in vuln_summary or 'dos' in vuln_summary else 0,
                'has_csrf': 1 if 'csrf' in vuln_summary or 'cross-site request' in vuln_summary else 0,
                'has_path_traversal': 1 if 'path traversal' in vuln_summary or 'directory traversal' in vuln_summary else 0,
                'has_file_inclusion': 1 if 'file inclusion' in vuln_summary or 'inclusion de fichier' in vuln_summary else 0,
                'has_encryption': 1 if 'encryption' in vuln_summary or 'chiffrement' in vuln_summary else 0,
                'has_ssl': 1 if 'ssl' in vuln_summary or 'tls' in vuln_summary else 0,
                'has_mitm': 1 if 'man in the middle' in vuln_summary or 'mitm' in vuln_summary or 'homme du milieu' in vuln_summary else 0,
                'has_bypass': 1 if 'bypass' in vuln_summary or 'contournement' in vuln_summary else 0,
                'has_backdoor': 1 if 'backdoor' in vuln_summary or 'porte dérobée' in vuln_summary else 0,
                'has_zero_day': 1 if 'zero day' in vuln_summary or 'zero-day' in vuln_summary or '0-day' in vuln_summary else 0,

                # Métriques existantes
                'risk_level': row['Risk Level Numeric'],
                'impact_level': row['Impact Level Numeric']
            }

           features.append(feature_dict)

        # Conversion en DataFrame
        X_features = pd.DataFrame(features)
        print(f"   ✓ Caractéristiques basées sur les règles extraites: {X_features.shape[1]} dimensions")

         # Vérifier et traiter les valeurs NaN AVANT le scaling
        if X_features.isna().any().any():
           print("   ⚠️ Valeurs manquantes détectées dans les caractéristiques. Remplacement par 0...")
           X_features = X_features.fillna(0)  # Remplacer toutes les valeurs NaN par 0

        # Application du scaling
        if self.config['scaling_method'] == 'standard':
            self.scaler = StandardScaler()
        elif self.config['scaling_method'] == 'minmax':
            self.scaler = MinMaxScaler()
        elif self.config['scaling_method'] == 'robust':
            self.scaler = RobustScaler()
        else:
            self.scaler = StandardScaler()

        X_features_scaled = self.scaler.fit_transform(X_features)
        print(f"   ✓ Scaling appliqué: {self.config['scaling_method']}")

        # Enregistrement du scaler
        joblib.dump(self.scaler, 'models/feature_scaler.pkl')



         # Lorsque vous arrivez à SelectKBest, utilisez la variable df mise à jour:
        if self.config['feature_selection'] and X_features.shape[1] > self.config['n_features']:
            print(f"   - Sélection des {self.config['n_features']} meilleures caractéristiques...")

            self.feature_selector = SelectKBest(f_classif, k=self.config['n_features'])
        # Utiliser df['Risk Level Numeric'] qui a déjà été nettoyé des NaN
            X_features_selected = self.feature_selector.fit_transform(X_features_scaled, df['Risk Level Numeric'])

            # Enregistrement du sélecteur
            joblib.dump(self.feature_selector, 'models/feature_selector.pkl')

            print(f"   ✓ Sélection de caractéristiques appliquée: {X_features_selected.shape[1]} dimensions retenues")

            # Récupérer et afficher les noms des caractéristiques sélectionnées
            selected_indices = self.feature_selector.get_support(indices=True)
            selected_features = X_features.columns[selected_indices].tolist()
            print(f"   📋 Caractéristiques sélectionnées: {selected_features}")

            return X_features_selected, X_text, selected_features
        else:
            return X_features_scaled, X_text, X_features.columns.tolist()

    def apply_sampling(self, X, y):
        """Applique des techniques de sur-échantillonnage ou sous-échantillonnage"""
        if not self.config['apply_smote']:
            return X, y

        print("\n⚖️ Application de techniques d'équilibrage des classes...")

        # Affichage de la distribution initiale
        class_counts = pd.Series(y).value_counts().sort_index()
        print(f"   - Distribution initiale des classes: {class_counts.to_dict()}")

        # Application de SMOTE
        smote = SMOTE(random_state=42)
        X_resampled, y_resampled = smote.fit_resample(X, y)

        # Affichage de la distribution après SMOTE
        class_counts_after = pd.Series(y_resampled).value_counts().sort_index()
        print(f"   ✓ Distribution après SMOTE: {class_counts_after.to_dict()}")

        return X_resampled, y_resampled

#EXTRACTION DES CARACTÉRISTIQUES

In [ ]:
def extract_features(self, df):
    """Extrait des caractéristiques pour l'analyse de vulnérabilité"""
    print("\n🔧 Extraction des caractéristiques...")

    # Caractéristiques textuelles
    if 'Combined_Text' in df.columns:
        text_column = 'Combined_Text'
    else:
        text_column = 'Vulnerability Summary'

    print(f"   - Utilisation de la colonne '{text_column}' pour l'extraction textuelle")

    # Vectorisation des descriptions
    tfidf = TfidfVectorizer(max_features=1000, stop_words=self.stop_words)
    X_text = tfidf.fit_transform(df[text_column])
    print(f"   ✓ Caractéristiques textuelles extraites: {X_text.shape[1]} dimensions")

    # Enregistrement du vectoriseur
    joblib.dump(tfidf, 'models/tfidf_vectorizer.pkl')

    # Extraction de caractéristiques basées sur les règles
    features = []
    print("   - Extraction des caractéristiques basées sur les règles...")

    for idx, row in tqdm(df.iterrows(), total=df.shape[0], desc="   🔄 Traitement des entrées"):
        affected_systems = str(row['Affected Systems']).lower()
        vuln_summary = str(row['Vulnerability Summary']).lower()

        feature_dict = {
            # Systèmes affectés
            'has_windows': 1 if 'windows' in affected_systems else 0,
            'has_linux': 1 if 'linux' in affected_systems else 0,
            'has_macos': 1 if ('macos' in affected_systems or 'mac os' in affected_systems) else 0,
            'has_android': 1 if 'android' in affected_systems else 0,
            'has_ios': 1 if ('ios' in affected_systems or 'iphone' in affected_systems or 'ipad' in affected_systems) else 0,
            'has_wordpress': 1 if 'wordpress' in affected_systems else 0,
            'has_joomla': 1 if 'joomla' in affected_systems else 0,
            'has_drupal': 1 if 'drupal' in affected_systems else 0,
            'has_oracle': 1 if 'oracle' in affected_systems else 0,
            'has_microsoft': 1 if 'microsoft' in affected_systems else 0,
            'has_adobe': 1 if 'adobe' in affected_systems else 0,
            'has_plugin': 1 if 'plugin' in affected_systems or 'extension' in affected_systems else 0,
            'has_server': 1 if 'server' in affected_systems or 'serveur' in affected_systems else 0,
            'has_web': 1 if 'web' in affected_systems else 0,
            'has_mobile': 1 if 'mobile' in affected_systems else 0,
            'has_iot': 1 if 'iot' in affected_systems or 'internet of things' in affected_systems else 0,
            'has_cloud': 1 if 'cloud' in affected_systems or 'aws' in affected_systems or 'azure' in affected_systems else 0,

            # Types de vulnérabilités
            'has_injection': 1 if 'injection' in vuln_summary else 0,
            'has_xss': 1 if 'xss' in vuln_summary or 'cross-site' in vuln_summary or 'script' in vuln_summary else 0,
            'has_sql': 1 if 'sql' in vuln_summary else 0,
            'has_authentication': 1 if 'authentification' in vuln_summary or 'authentication' in vuln_summary else 0,
            'has_authorization': 1 if 'autorisation' in vuln_summary or 'authorization' in vuln_summary else 0,
            'has_privilege': 1 if 'privilège' in vuln_summary or 'privilege' in vuln_summary else 0,
            'has_code_execution': 1 if 'exécution de code' in vuln_summary or 'code execution' in vuln_summary else 0,
            'has_arbitrary_code': 1 if 'arbitraire' in vuln_summary or 'arbitrary' in vuln_summary else 0,
            'has_buffer': 1 if 'buffer' in vuln_summary or 'tampon' in vuln_summary else 0,
            'has_overflow': 1 if 'overflow' in vuln_summary or 'débordement' in vuln_summary else 0,
            'has_dos': 1 if 'déni de service' in vuln_summary or 'denial of service' in vuln_summary or 'dos' in vuln_summary else 0,
            'has_csrf': 1 if 'csrf' in vuln_summary or 'cross-site request' in vuln_summary else 0,
            'has_path_traversal': 1 if 'path traversal' in vuln_summary or 'directory traversal' in vuln_summary else 0,
            'has_file_inclusion': 1 if 'file inclusion' in vuln_summary or 'inclusion de fichier' in vuln_summary else 0,
            'has_encryption': 1 if 'encryption' in vuln_summary or 'chiffrement' in vuln_summary else 0,
            'has_ssl': 1 if 'ssl' in vuln_summary or 'tls' in vuln_summary else 0,
            'has_mitm': 1 if 'man in the middle' in vuln_summary or 'mitm' in vuln_summary or 'homme du milieu' in vuln_summary else 0,
            'has_bypass': 1 if 'bypass' in vuln_summary or 'contournement' in vuln_summary else 0,
            'has_backdoor': 1 if 'backdoor' in vuln_summary or 'porte dérobée' in vuln_summary else 0,
            'has_zero_day': 1 if 'zero day' in vuln_summary or 'zero-day' in vuln_summary or '0-day' in vuln_summary else 0,

            # Métriques existantes
            'risk_level': row['Risk Level Numeric'],
            'impact_level': row['Impact Level Numeric']
        }

        features.append(feature_dict)

    # Conversion en DataFrame
    X_features = pd.DataFrame(features)
    print(f"   ✓ Caractéristiques basées sur les règles extraites: {X_features.shape[1]} dimensions")

    # Application du scaling
    if self.config['scaling_method'] == 'standard':
        self.scaler = StandardScaler()
    elif self.config['scaling_method'] == 'minmax':
        self.scaler = MinMaxScaler()
    elif self.config['scaling_method'] == 'robust':
        self.scaler = RobustScaler()
    else:
        self.scaler = StandardScaler()

    X_features_scaled = self.scaler.fit_transform(X_features)
    print(f"   ✓ Scaling appliqué: {self.config['scaling_method']}")

    # Enregistrement du scaler
    joblib.dump(self.scaler, 'models/feature_scaler.pkl')

    # Sélection des caractéristiques si configurée
    if self.config['feature_selection'] and X_features.shape[1] > self.config['n_features']:
        print(f"   - Sélection des {self.config['n_features']} meilleures caractéristiques...")

        # Utilisation de SelectKBest avec f_classif
        self.feature_selector = SelectKBest(f_classif, k=self.config['n_features'])
        X_features_selected = self.feature_selector.fit_transform(X_features_scaled, df['Risk Level Numeric'])

        # Enregistrement du sélecteur
        joblib.dump(self.feature_selector, 'models/feature_selector.pkl')

        print(f"   ✓ Sélection de caractéristiques appliquée: {X_features_selected.shape[1]} dimensions retenues")

        # Récupérer et afficher les noms des caractéristiques sélectionnées
        selected_indices = self.feature_selector.get_support(indices=True)
        selected_features = X_features.columns[selected_indices].tolist()
        print(f"   📋 Caractéristiques sélectionnées: {selected_features}")

        return X_features_selected, X_text, selected_features
    else:
        return X_features_scaled, X_text, X_features.columns.tolist()

def apply_sampling(self, X, y):
    """Applique des techniques de sur-échantillonnage ou sous-échantillonnage"""
    if not self.config['apply_smote']:
        return X, y

    print("\n⚖️ Application de techniques d'équilibrage des classes...")

    # Affichage de la distribution initiale
    class_counts = pd.Series(y).value_counts().sort_index()
    print(f"   - Distribution initiale des classes: {class_counts.to_dict()}")

    # Application de SMOTE
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)

    # Affichage de la distribution après SMOTE
    class_counts_after = pd.Series(y_resampled).value_counts().sort_index()
    print(f"   ✓ Distribution après SMOTE: {class_counts_after.to_dict()}")

    return X_resampled, y_resampled

In [ ]:
def train_and_evaluate(self, X, y, feature_names=None):
    """Entraîne et évalue tous les modèles configurés"""
    print("\n🤖 Entraînement et évaluation des modèles...")

    # Initialisation des modèles
    self.models = self.initialize_models()

    if not self.models:
        print("⚠️ Aucun modèle configuré pour l'entraînement")
        return

    # Division des données
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    print(f"   - Données divisées: {X_train.shape[0]} exemples d'entraînement, {X_test.shape[0]} exemples de test")

    # Entraînement et évaluation de chaque modèle
    results = {}

    for name, model in self.models.items():
        start_time = time.time()
        print(f"\n   🔄 Entraînement du modèle: {name}...")

        # Vérification si Grid Search est activé
        grid_search_config = self.models_config.get(name, {}).get('grid_search', False)
        grid_params = self.models_config.get(name, {}).get('grid_params', {})

        if grid_search_config and grid_params:
            print(f"      - Application de Grid Search avec {len(grid_params)} paramètres")
            grid_search = GridSearchCV(
                model,
                grid_params,
                cv=StratifiedKFold(n_splits=5),
                scoring='f1_weighted',
                n_jobs=-1
            )
            grid_search.fit(X_train, y_train)
            model = grid_search.best_estimator_
            print(f"      ✓ Meilleurs paramètres: {grid_search.best_params_}")
        else:
            model.fit(X_train, y_train)

        # Prédictions
        y_pred = model.predict(X_test)

        # Calcul des métriques
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')

        # Temps d'entraînement
        training_time = time.time() - start_time

        # Stockage des résultats
        results[name] = {
            'model': model,
            'accuracy': accuracy,
            'f1_score': f1,
            'training_time': training_time
        }

        # Affichage du rapport de classification
        print(f"      ✓ Entraînement terminé en {training_time:.2f} secondes")
        print(f"      ✓ Accuracy: {accuracy:.4f}")
        print(f"      ✓ F1-Score: {f1:.4f}")
        print("\n      📊 Rapport de classification:")
        print(classification_report(y_test, y_pred))

        # Matrice de confusion
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=['Faible', 'Modéré', 'Important', 'Critique'],
                    yticklabels=['Faible', 'Modéré', 'Important', 'Critique'])
        plt.title(f'Matrice de confusion - {name}')
        plt.ylabel('Valeur réelle')
        plt.xlabel('Valeur prédite')
        plt.tight_layout()
        plt.savefig(f'models/confusion_matrix_{name}.png')
        plt.close()

        # Importance des caractéristiques si disponible
        if hasattr(model, 'feature_importances_') and feature_names is not None:
            feature_importance = pd.DataFrame({
                'Feature': feature_names,
                'Importance': model.feature_importances_
            }).sort_values('Importance', ascending=False)

            # Ploter les 20 premières caractéristiques les plus importantes
            plt.figure(figsize=(10, 8))
            sns.barplot(x='Importance', y='Feature', data=feature_importance.head(20))
            plt.title(f'Importance des caractéristiques - {name}')
            plt.tight_layout()
            plt.savefig(f'models/feature_importance_{name}.png')
            plt.close()

            # Sauvegarder l'importance des caractéristiques pour le meilleur modèle
            if f1 > self.best_score:
                self.feature_importance = feature_importance

        # Sauvegarder le modèle
        joblib.dump(model, f'models/{name}_model.pkl')

        # Mise à jour du meilleur modèle
        if f1 > self.best_score:
            self.best_score = f1
            self.best_model_name = name
            self.best_model = model

    # Rapport final
    print("\n📋 Résumé des performances des modèles:")
    results_df = pd.DataFrame({
        'Modèle': list(results.keys()),
        'Accuracy': [results[model]['accuracy'] for model in results],
        'F1-Score': [results[model]['f1_score'] for model in results],
        'Temps (s)': [results[model]['training_time'] for model in results]
    }).sort_values('F1-Score', ascending=False)

    print(results_df)

    # Enregistrement du meilleur modèle
    if self.best_model is not None:
        print(f"\n🏆 Meilleur modèle: {self.best_model_name} (F1-Score: {self.best_score:.4f})")
        joblib.dump(self.best_model, 'models/best_model.pkl')

        # Sauvegarde du meilleur nom de modèle
        with open('models/best_model_name.txt', 'w') as f:
            f.write(self.best_model_name)

    return results

In [ ]:
def train_and_evaluate(self, X, y, feature_names=None):
    """Entraîne et évalue tous les modèles configurés"""
    print("\n🤖 Entraînement et évaluation des modèles...")

    # Initialisation des modèles
    self.models = self.initialize_models()

    if not self.models:
        print("⚠️ Aucun modèle configuré pour l'entraînement")
        return

    # Division des données
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    print(f"   - Données divisées: {X_train.shape[0]} exemples d'entraînement, {X_test.shape[0]} exemples de test")

    # Entraînement et évaluation de chaque modèle
    results = {}

    for name, model in self.models.items():
        start_time = time.time()
        print(f"\n   🔄 Entraînement du modèle: {name}...")

        # Vérification si Grid Search est activé
        grid_search_config = self.models_config.get(name, {}).get('grid_search', False)
        grid_params = self.models_config.get(name, {}).get('grid_params', {})

        if grid_search_config and grid_params:
            print(f"      - Application de Grid Search avec {len(grid_params)} paramètres")
            grid_search = GridSearchCV(
                model,
                grid_params,
                cv=StratifiedKFold(n_splits=5),
                scoring='f1_weighted',
                n_jobs=-1
            )
            grid_search.fit(X_train, y_train)
            model = grid_search.best_estimator_
            print(f"      ✓ Meilleurs paramètres: {grid_search.best_params_}")
        else:
            model.fit(X_train, y_train)

        # Prédictions
        y_pred = model.predict(X_test)

        # Calcul des métriques
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')

        # Temps d'entraînement
        training_time = time.time() - start_time

        # Stockage des résultats
        results[name] = {
            'model': model,
            'accuracy': accuracy,
            'f1_score': f1,
            'training_time': training_time
        }

        # Affichage du rapport de classification
        print(f"      ✓ Entraînement terminé en {training_time:.2f} secondes")
        print(f"      ✓ Accuracy: {accuracy:.4f}")
        print(f"      ✓ F1-Score: {f1:.4f}")
        print("\n      📊 Rapport de classification:")
        print(classification_report(y_test, y_pred))

        # Matrice de confusion
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=['Faible', 'Modéré', 'Important', 'Critique'],
                    yticklabels=['Faible', 'Modéré', 'Important', 'Critique'])
        plt.title(f'Matrice de confusion - {name}')
        plt.ylabel('Valeur réelle')
        plt.xlabel('Valeur prédite')
        plt.tight_layout()
        plt.savefig(f'models/confusion_matrix_{name}.png')
        plt.close()

        # Importance des caractéristiques si disponible
        if hasattr(model, 'feature_importances_') and feature_names is not None:
            feature_importance = pd.DataFrame({
                'Feature': feature_names,
                'Importance': model.feature_importances_
            }).sort_values('Importance', ascending=False)

            # Ploter les 20 premières caractéristiques les plus importantes
            plt.figure(figsize=(10, 8))
            sns.barplot(x='Importance', y='Feature', data=feature_importance.head(20))
            plt.title(f'Importance des caractéristiques - {name}')
            plt.tight_layout()
            plt.savefig(f'models/feature_importance_{name}.png')
            plt.close()

            # Sauvegarder l'importance des caractéristiques pour le meilleur modèle
            if f1 > self.best_score:
                self.feature_importance = feature_importance

        # Sauvegarder le modèle
        joblib.dump(model, f'models/{name}_model.pkl')

        # Mise à jour du meilleur modèle
        if f1 > self.best_score:
            self.best_score = f1
            self.best_model_name = name
            self.best_model = model

    # Rapport final
    print("\n📋 Résumé des performances des modèles:")
    results_df = pd.DataFrame({
        'Modèle': list(results.keys()),
        'Accuracy': [results[model]['accuracy'] for model in results],
        'F1-Score': [results[model]['f1_score'] for model in results],
        'Temps (s)': [results[model]['training_time'] for model in results]
    }).sort_values('F1-Score', ascending=False)

    print(results_df)

    # Enregistrement du meilleur modèle
    if self.best_model is not None:
        print(f"\n🏆 Meilleur modèle: {self.best_model_name} (F1-Score: {self.best_score:.4f})")
        joblib.dump(self.best_model, 'models/best_model.pkl')

        # Sauvegarde du meilleur nom de modèle
        with open('models/best_model_name.txt', 'w') as f:
            f.write(self.best_model_name)

    return results

#SECTION 2: ENTRAÎNEMENT DE MULTIPLES MODÈLES


---



In [ ]:
class MultiModelTrainer:
    """Classe pour l'entraînement et l'évaluation de multiples modèles"""

    def __init__(self, models_config=None):
        self.models_config = models_config or {
            'random_forest': {
                'enabled': True,
                'params': {
                    'n_estimators': 100,
                    'max_depth': None,
                    'min_samples_split': 2,
                    'random_state': 42
                },
                'grid_search': False,
                'grid_params': {
                    'n_estimators': [50, 100, 200],
                    'max_depth': [None, 10, 20],
                    'min_samples_split': [2, 5, 10]
                }
            },
            'xgboost': {
                'enabled': True,
                'params': {
                    'n_estimators': 100,
                    'learning_rate': 0.1,
                    'max_depth': 5,
                    'random_state': 42
                },
                'grid_search': False,
                'grid_params': {
                    'n_estimators': [50, 100, 200],
                    'learning_rate': [0.01, 0.1, 0.2],
                    'max_depth': [3, 5, 7]
                }
            },
            'gradient_boosting': {
                'enabled': True,
                'params': {
                    'n_estimators': 100,
                    'learning_rate': 0.1,
                    'max_depth': 3,
                    'random_state': 42
                },
                'grid_search': False,
                'grid_params': {
                    'n_estimators': [50, 100, 200],
                    'learning_rate': [0.01, 0.1, 0.2],
                    'max_depth': [3, 5, 7]
                }
            },
            'logistic_regression': {
                'enabled': True,
                'params': {
                    'C': 1.0,
                    'random_state': 42
                },
                'grid_search': False
            },
            'voting_ensemble': {
                'enabled': True,
                'estimators': ['random_forest', 'xgboost', 'gradient_boosting'],
                'voting': 'soft'
            }
        }

        self.models = {}
        self.best_model_name = None
        self.best_model = None
        self.best_score = 0
        self.feature_importance = None

    def initialize_models(self):
        """Initialise tous les modèles configurés"""
        models = {}

        # Random Forest
        if self.models_config['random_forest']['enabled']:
            models['random_forest'] = RandomForestClassifier(**self.models_config['random_forest']['params'])

        # XGBoost
        if self.models_config['xgboost']['enabled']:
            models['xgboost'] = XGBClassifier(**self.models_config['xgboost']['params'])

        # Gradient Boosting
        if self.models_config['gradient_boosting']['enabled']:
            models['gradient_boosting'] = GradientBoostingClassifier(**self.models_config['gradient_boosting']['params'])

        # Logistic Regression
        if self.models_config['logistic_regression']['enabled']:
            models['logistic_regression'] = LogisticRegression(**self.models_config['logistic_regression']['params'])

        # Ajout du modèle d'ensemble si configuré
        if self.models_config['voting_ensemble']['enabled']:
            estimators = []
            for est_name in self.models_config['voting_ensemble']['estimators']:
                if est_name in models:
                    estimators.append((est_name, models[est_name]))

            if estimators:
                models['voting_ensemble'] = VotingClassifier(
                    estimators=estimators,
                    voting=self.models_config['voting_ensemble']['voting']
                )

        return models

    def train_and_evaluate(self, X, y, feature_names=None):
        """Entraîne et évalue tous les modèles configurés"""
        print("\n🤖 Entraînement et évaluation des modèles...")

        # Initialisation des modèles
        self.models = self.initialize_models()

        if not self.models:
            print("⚠️ Aucun modèle configuré pour l'entraînement")
            return

        # Division des données
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        print(f"   - Données divisées: {X_train.shape[0]} exemples d'entraînement, {X_test.shape[0]} exemples de test")

        # Entraînement et évaluation de chaque modèle
        results = {}

        for name, model in self.models.items():
            start_time = time.time()
            print(f"\n   🔄 Entraînement du modèle: {name}...")

            # Vérification si Grid Search est activé
            grid_search_config = self.models_config.get(name, {}).get('grid_search', False)
            grid_params = self.models_config.get(name, {}).get('grid_params', {})

            if grid_search_config and grid_params:
                print(f"      - Application de Grid Search avec {len(grid_params)} paramètres")
                grid_search = GridSearchCV(
                    model,
                    grid_params,
                    cv=StratifiedKFold(n_splits=5),
                    scoring='f1_weighted',
                    n_jobs=-1
                )
                grid_search.fit(X_train, y_train)
                model = grid_search.best_estimator_
                print(f"      ✓ Meilleurs paramètres: {grid_search.best_params_}")
            else:
                model.fit(X_train, y_train)

            # Prédictions
            y_pred = model.predict(X_test)

            # Calcul des métriques
            accuracy = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')

            # Temps d'entraînement
            training_time = time.time() - start_time

            # Stockage des résultats
            results[name] = {
                'model': model,
                'accuracy': accuracy,
                'f1_score': f1,
                'training_time': training_time
            }

            # Affichage du rapport de classification
            print(f"      ✓ Entraînement terminé en {training_time:.2f} secondes")
            print(f"      ✓ Accuracy: {accuracy:.4f}")
            print(f"      ✓ F1-Score: {f1:.4f}")
            print("\n      📊 Rapport de classification:")
            print(classification_report(y_test, y_pred))

            # Matrice de confusion
            cm = confusion_matrix(y_test, y_pred)
            plt.figure(figsize=(8, 6))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                        xticklabels=['Faible', 'Modéré', 'Important', 'Critique'],
                        yticklabels=['Faible', 'Modéré', 'Important', 'Critique'])
            plt.title(f'Matrice de confusion - {name}')
            plt.ylabel('Valeur réelle')
            plt.xlabel('Valeur prédite')
            plt.tight_layout()
            plt.savefig(f'models/confusion_matrix_{name}.png')
            plt.close()

            # Importance des caractéristiques si disponible
            if hasattr(model, 'feature_importances_') and feature_names is not None:
                feature_importance = pd.DataFrame({
                    'Feature': feature_names,
                    'Importance': model.feature_importances_
                }).sort_values('Importance', ascending=False)

                # Ploter les 20 premières caractéristiques les plus importantes
                plt.figure(figsize=(10, 8))
                sns.barplot(x='Importance', y='Feature', data=feature_importance.head(20))
                plt.title(f'Importance des caractéristiques - {name}')
                plt.tight_layout()
                plt.savefig(f'models/feature_importance_{name}.png')
                plt.close()

                # Sauvegarder l'importance des caractéristiques pour le meilleur modèle
                if f1 > self.best_score:
                    self.feature_importance = feature_importance

            # Sauvegarder le modèle
            joblib.dump(model, f'models/{name}_model.pkl')

            # Mise à jour du meilleur modèle
            if f1 > self.best_score:
                self.best_score = f1
                self.best_model_name = name
                self.best_model = model

        # Rapport final
        print("\n📋 Résumé des performances des modèles:")
        results_df = pd.DataFrame({
            'Modèle': list(results.keys()),
            'Accuracy': [results[model]['accuracy'] for model in results],
            'F1-Score': [results[model]['f1_score'] for model in results],
            'Temps (s)': [results[model]['training_time'] for model in results]
        }).sort_values('F1-Score', ascending=False)

        print(results_df)

        # Enregistrement du meilleur modèle
        if self.best_model is not None:
            print(f"\n🏆 Meilleur modèle: {self.best_model_name} (F1-Score: {self.best_score:.4f})")
            joblib.dump(self.best_model, 'models/best_model.pkl')

            # Sauvegarde du meilleur nom de modèle
            with open('models/best_model_name.txt', 'w') as f:
                f.write(self.best_model_name)

        return results

#SECTION 3: DÉTECTION D'ANOMALIES


In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor  # Attention: c'est dans sklearn.neighbors, pas sklearn.ensemble
from sklearn.decomposition import PCA

In [ ]:
class AnomalyDetector:
    """Classe pour la détection d'anomalies et l'identification de vulnérabilités rares"""

    def __init__(self, config=None):
        self.config = config or {
            'isolation_forest': {
                'enabled': True,
                'params': {
                    'n_estimators': 100,
                    'contamination': 0.05,
                    'random_state': 42
                }
            },
            'local_outlier_factor': {
                'enabled': True,
                'params': {
                    'n_neighbors': 20,
                    'contamination': 0.05
                }
            },
            'one_class_svm': {
                'enabled': False,
                'params': {
                    'kernel': 'rbf',
                    'nu': 0.05,
                    'gamma': 'scale'
                }
            }
        }

        self.models = {}
        self.best_model = None
        self.best_model_name = None

    def train_anomaly_models(self, X):
        """Entraîne les modèles de détection d'anomalies"""
        print("\n🔍 Entraînement des modèles de détection d'anomalies...")

    # Isolation Forest
        if 'isolation_forest' in self.config and self.config['isolation_forest']['enabled']:
            print("   - Entraînement d'Isolation Forest...")
            iso_forest = IsolationForest(**self.config['isolation_forest']['params'])
            iso_forest.fit(X)
            self.models['isolation_forest'] = iso_forest
            joblib.dump(iso_forest, 'models/isolation_forest_model.pkl')
            print("   ✓ Modèle Isolation Forest enregistré")

        # Définir comme meilleur modèle par défaut si c'est le premier
        if self.best_model is None:
            self.best_model = iso_forest
            self.best_model_name = 'isolation_forest'

    # Local Outlier Factor
        if 'local_outlier_factor' in self.config and self.config['local_outlier_factor']['enabled']:
            print("   - Entraînement de Local Outlier Factor...")
            lof = LocalOutlierFactor(**self.config['local_outlier_factor']['params'])
        # LOF a besoin d'être ajusté et ne peut pas être utilisé directement pour prédire de nouvelles données
        # Nous allons donc l'entraîner sur les données d'entraînement et sauvegarder le modèle
            lof.fit_predict(X)
            self.models['local_outlier_factor'] = lof
            joblib.dump(lof, 'models/local_outlier_factor_model.pkl')
            print("   ✓ Modèle Local Outlier Factor enregistré")

    # One-Class SVM - vérifier si la clé existe
        if 'one_class_svm' in self.config and self.config['one_class_svm']['enabled']:
            try:
               from sklearn.svm import OneClassSVM
               print("   - Entraînement de One-Class SVM...")
               ocsvm = OneClassSVM(**self.config['one_class_svm']['params'])
               ocsvm.fit(X)
               self.models['one_class_svm'] = ocsvm
               joblib.dump(ocsvm, 'models/one_class_svm_model.pkl')
               print("   ✓ Modèle One-Class SVM enregistré")
            except Exception as e:
               print(f"   ⚠️ Erreur lors de l'entraînement de One-Class SVM: {e}")

    # Enregistrement du meilleur modèle (par défaut, Isolation Forest)
        if self.best_model is not None:
          joblib.dump(self.best_model, 'models/best_anomaly_model.pkl')
          with open('models/best_anomaly_model_name.txt', 'w') as f:
               f.write(self.best_model_name)

        return self.models

    def visualize_anomaly_detection(self, X, model_name='isolation_forest'):
        """Visualise les résultats de la détection d'anomalies"""
        if model_name not in self.models:
            print(f"⚠️ Modèle {model_name} non trouvé.")
            return

        print(f"\n📊 Visualisation des anomalies détectées par {model_name}...")

        # Réduire la dimensionnalité pour visualisation
        pca = PCA(n_components=2)
        X_2d = pca.fit_transform(X)

        # Prédiction des anomalies
        if model_name == 'isolation_forest':
            y_pred = self.models[model_name].predict(X)
        elif model_name == 'local_outlier_factor':
            # Pour LOF, nous utilisons les scores de nouveauté
            y_pred = self.models[model_name].fit_predict(X)
        else:
            y_pred = self.models[model_name].predict(X)

        # Conversion des valeurs prédites (-1 pour anomalie, 1 pour normal)
        anomalies = y_pred == -1

        # Création d'un dataframe pour visualisation
        df_viz = pd.DataFrame({
            'PC1': X_2d[:, 0],
            'PC2': X_2d[:, 1],
            'Anomalie': anomalies
        })

        # Calcul du pourcentage d'anomalies
        anomaly_percentage = (anomalies.sum() / len(anomalies)) * 100

        # Visualisation
        plt.figure(figsize=(10, 8))
        sns.scatterplot(x='PC1', y='PC2', hue='Anomalie', data=df_viz, palette={False: 'blue', True: 'red'})
        plt.title(f'Détection d\'anomalies - {model_name}\n{anomalies.sum()} anomalies détectées ({anomaly_percentage:.2f}%)')
        plt.legend(['Normal', 'Anomalie'])
        plt.savefig(f'models/anomaly_detection_{model_name}.png')
        plt.close()

        print(f"   ✓ {anomalies.sum()} anomalies détectées ({anomaly_percentage:.2f}%)")
        return df_viz



def create_directory_structure():
    """Crée la structure de répertoires nécessaire pour le projet"""
    print("\n📂 Création de la structure de répertoires...")

    # Liste des répertoires à créer
    directories = [
        'models',
        'data',
        'reports',
        'logs',
        'static',
        'static/img',
        'templates'
    ]

    # Création des répertoires
    for directory in directories:
        os.makedirs(directory, exist_ok=True)

    print("   ✓ Structure de répertoires créée")

def generate_security_report(result, output_format='json'):
    """Génère un rapport de sécurité dans le format spécifié"""
    print("\n📝 Génération du rapport de sécurité...")

    if output_format == 'json':
        # Enregistrement du rapport au format JSON
        report_file = f"reports/security_report_{result['url'].replace('://', '_').replace('/', '_').replace('.', '_')}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(report_file, 'w') as f:
            json.dump(result, f, indent=4)
        print(f"   ✓ Rapport JSON enregistré: {report_file}")
        return report_file
    elif output_format == 'html':
        # Création d'un rapport HTML simple
        report_file = f"reports/security_report_{result['url'].replace('://', '_').replace('/', '_').replace('.', '_')}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"

        # Template HTML basique
        html_template = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>Rapport de sécurité - {result['url']}</title>
            <style>
                body {{ font-family: Arial, sans-serif; margin: 20px; }}
                h1, h2, h3 {{ color: #333; }}
                .header {{ background-color: #f8f9fa; padding: 20px; border-radius: 5px; margin-bottom: 20px; }}
                .risk-score {{ font-size: 24px; font-weight: bold; }}
                .risk-critical {{ color: #dc3545; }}
                .risk-important {{ color: #fd7e14; }}
                .risk-moderate {{ color: #ffc107; }}
                .risk-low {{ color: #28a745; }}
                .vulnerability {{ margin: 10px 0; padding: 15px; border-radius: 5px; }}
                .vulnerability-critical {{ background-color: #f8d7da; border-left: 5px solid #dc3545; }}
                .vulnerability-important {{ background-color: #fff3cd; border-left: 5px solid #fd7e14; }}
                .vulnerability-moderate {{ background-color: #fff3cd; border-left: 5px solid #ffc107; }}
                .recommendation {{ margin: 10px 0; padding: 15px; background-color: #e9ecef; border-radius: 5px; }}
            </style>
        </head>
        <body>
            <div class="header">
                <h1>Rapport de sécurité web</h1>
                <p><strong>URL analysée:</strong> {result['url']}</p>
                <p><strong>Date d'analyse:</strong> {result['scan_date']}</p>
                <p class="risk-score {
                    'risk-critical' if result['risk_level_text'] == 'Critique' else
                    'risk-important' if result['risk_level_text'] == 'Important' else
                    'risk-moderate' if result['risk_level_text'] == 'Modéré' else
                    'risk-low'
                }">
                    Niveau de risque: {result['risk_level_text']} ({result['risk_score']}%)
                </p>
                <p><strong>Anomalie détectée:</strong> {'Oui' if result['is_anomaly'] else 'Non'}</p>
            </div>

            <h2>Vulnérabilités détectées ({result['vulnerability_count']})</h2>
        """

        # Ajout des vulnérabilités
        if result['vulnerability_count'] > 0:
            for vuln in result['vulnerabilities']:
                severity_class = 'vulnerability-critical' if vuln['severity'] == 'Critique' else \
                                'vulnerability-important' if vuln['severity'] == 'Important' else \
                                'vulnerability-moderate'
                html_template += f"""
                <div class="vulnerability {severity_class}">
                    <h3>{vuln['name']}</h3>
                    <p>{vuln['description']}</p>
                    <p><strong>Sévérité:</strong> {vuln['severity']}</p>
                </div>
                """
        else:
            html_template += "<p>Aucune vulnérabilité détectée.</p>"

        # Ajout des recommandations
        html_template += "<h2>Recommandations de sécurité</h2>"

        if result['recommendations']:
            for rec in result['recommendations']:
                html_template += f"""
                <div class="recommendation">
                    <h3>{rec['title']}</h3>
                    <p>{rec['description']}</p>
                    <p><strong>Priorité:</strong> {rec['priority']}</p>
                </div>
                """
        else:
            html_template += "<p>Aucune recommandation spécifique.</p>"

        # Ajout des probabilités
        html_template += """
            <h2>Probabilités par niveau de risque</h2>
            <table border="1" cellpadding="5" cellspacing="0">
                <tr>
                    <th>Niveau</th>
                    <th>Probabilité</th>
                </tr>
        """

        for level, prob in result['probability'].items():
            html_template += f"""
                <tr>
                    <td>{level}</td>
                    <td>{prob:.2f}</td>
                </tr>
            """

        html_template += """
            </table>

            <div style="margin-top: 30px; text-align: center; color: #6c757d;">
                <p>Rapport généré par le système de prédiction de vulnérabilités web.</p>
                <p>© 2025 Direction Générale de la Sécurité des Systèmes d'Information</p>
            </div>
        </body>
        </html>
        """

        # Enregistrement du rapport HTML
        with open(report_file, 'w') as f:
            f.write(html_template)

        print(f"   ✓ Rapport HTML enregistré: {report_file}")
        return report_file
    else:
        print(f"⚠️ Format de rapport non pris en charge: {output_format}")
        return None

def train_complete_pipeline(data_file, config=None):
    """Exécute le pipeline complet d'entraînement des modèles"""
    # Configuration par défaut
    default_config = {
        'preprocessing': {
            'text_cleaning': True,
            'handle_missing': 'fill_empty',
            'scaling_method': 'standard',
            'feature_selection': True,
            'n_features': 20,
            'stemming': True,
            'language': 'french',
            'apply_smote': False
        },
        'models': {
            'random_forest': {
                'enabled': True,
                'params': {'n_estimators': 100, 'random_state': 42},
                'grid_search': False
            },
            'xgboost': {
                'enabled': True,
                'params': {'n_estimators': 100, 'learning_rate': 0.1, 'random_state': 42},
                'grid_search': False
            },
            'gradient_boosting': {
                'enabled': True,
                'params': {'n_estimators': 100, 'learning_rate': 0.1, 'random_state': 42},
                'grid_search': False
            },
            'logistic_regression': {
                'enabled': True,
                'params': {'C': 1.0, 'random_state': 42},
                'grid_search': False
            },
            'voting_ensemble': {
                'enabled': True,
                'estimators': ['random_forest', 'xgboost', 'gradient_boosting'],
                'voting': 'soft'
            }
        },
        'anomaly_detection': {
            'isolation_forest': {
                'enabled': True,
                'params': {'n_estimators': 100, 'contamination': 0.05, 'random_state': 42}
            },
            'local_outlier_factor': {
                'enabled': True,
                'params': {'n_neighbors': 20, 'contamination': 0.05}
            }
        },
        'deep_learning': {
            'enabled': False,
            'model_type': 'mlp',
            'dense_units': [128, 64],
            'dropout_rate': 0.3,
            'learning_rate': 0.001,
            'batch_size': 32,
            'epochs': 50
        }
    }

    # Fusion de la configuration personnalisée avec la configuration par défaut
    if config:
        # Fonction récursive pour fusionner des dictionnaires imbriqués
        def merge_dicts(d1, d2):
            for k, v in d2.items():
                if k in d1 and isinstance(d1[k], dict) and isinstance(v, dict):
                    merge_dicts(d1[k], v)
                else:
                    d1[k] = v
            return d1

        config = merge_dicts(default_config.copy(), config)
    else:
        config = default_config

    # Création de la structure de répertoires
    create_directory_structure()

    # Prétraitement des données
    preprocessor = DataPreprocessor(config['preprocessing'])
    df = preprocessor.load_and_preprocess(data_file)

    # Extraction des caractéristiques
    X_features, X_text, selected_features = preprocessor.extract_features(df)

    # Préparation des données pour l'entraînement
    X = X_features  # Nous utilisons uniquement les caractéristiques basées sur les règles pour simplifier
    y = df['Risk Level Numeric'].values

    # Application de techniques de rééchantillonnage si configuré
    if config['preprocessing']['apply_smote']:
        X, y = preprocessor.apply_sampling(X, y)

    # Entraînement des modèles de classification
    model_trainer = MultiModelTrainer(config['models'])
    model_results = model_trainer.train_and_evaluate(X, y, selected_features)

    # Entraînement des modèles de détection d'anomalies
    anomaly_detector = AnomalyDetector(config['anomaly_detection'])
    anomaly_models = anomaly_detector.train_anomaly_models(X)

    # Visualisation des résultats de détection d'anomalies
    anomaly_viz = anomaly_detector.visualize_anomaly_detection(X)

    # Entraînement du modèle de deep learning si activé
    if config['deep_learning']['enabled']:
        dl_classifier = DeepLearningClassifier(config['deep_learning'])
        dl_history = dl_classifier.train(X, y)

    print("\n✅ Pipeline d'entraînement complet terminé!")
    return {
        'preprocessor': preprocessor,
        'model_trainer': model_trainer,
        'anomaly_detector': anomaly_detector,
        'best_model_name': model_trainer.best_model_name,
        'best_score': model_trainer.best_score
    }

def evaluate_website(url):
    """Fonction qui appelle l'analyseur de site web"""
    try:
        # Création de l'objet analyseur
        analyzer = WebsiteAnalyzer()

        # Analyse du site web
        result = analyzer.analyze_website(url)

        return result
    except Exception as e:
        return {
            'error': f'Erreur lors de l\'évaluation: {str(e)}',
            'url': url,
            'scan_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }



def handle_missing_values(self, df):
    """Gère les valeurs manquantes dans le DataFrame"""
    if self.config['handle_missing'] == 'drop':
        df = df.dropna()
    elif self.config['handle_missing'] == 'fill_empty':
        # Fill missing values with empty strings for string columns only
        string_cols = df.select_dtypes(include=['object']).columns
        df[string_cols] = df[string_cols].fillna('')
        # Impute numeric columns with 0
        numeric_cols = df.select_dtypes(include=np.number).columns
        df[numeric_cols] = df[numeric_cols].fillna(0)
    elif self.config['handle_missing'] == 'fill_median':
        for col in df.select_dtypes(include=np.number).columns:
             df[col] = df[col].fillna(df[col].median())
    return df

#SECTION 4: DEEP LEARNING POUR ANALYSE DE VULNÉRABILITÉS


In [ ]:
class DeepLearningClassifier:
    """Classe pour l'utilisation de modèles de deep learning pour la classification de vulnérabilités"""

    def __init__(self, config=None):
        self.config = config or {
            'enabled': False,
            'model_type': 'mlp',  # 'mlp', 'lstm', 'cnn'
            'embedding_size': 100,
            'lstm_units': 64,
            'dense_units': [128, 64],
            'dropout_rate': 0.3,
            'learning_rate': 0.001,
            'batch_size': 32,
            'epochs': 50,
            'early_stopping': True,
            'patience': 5
        }

        self.model = None
        self.history = None

    def build_mlp_model(self, input_shape, num_classes):
        """Construit un modèle MLP pour la classification"""
        model = Sequential()

        # Couche d'entrée
        model.add(Dense(self.config['dense_units'][0], activation='relu', input_shape=(input_shape,)))
        model.add(Dropout(self.config['dropout_rate']))

        # Couches cachées
        for units in self.config['dense_units'][1:]:
            model.add(Dense(units, activation='relu'))
            model.add(Dropout(self.config['dropout_rate']))

        # Couche de sortie
        model.add(Dense(num_classes, activation='softmax'))

        # Compilation
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config['learning_rate']),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        return model

    def train(self, X, y):
        """Entraîne le modèle de deep learning"""
        if not self.config['enabled']:
            print("⚠️ Le modèle de deep learning n'est pas activé dans la configuration.")
            return None

        print("\n🧠 Entraînement du modèle de deep learning...")

        # Division des données
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

        # Construction du modèle
        if self.config['model_type'] == 'mlp':
            self.model = self.build_mlp_model(X.shape[1], len(np.unique(y)))
        else:
            print(f"⚠️ Type de modèle non pris en charge: {self.config['model_type']}")
            return None

        # Callbacks
        callbacks = []
        if self.config['early_stopping']:
            early_stopping = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=self.config['patience'],
                restore_best_weights=True
            )
            callbacks.append(early_stopping)

        # Entraînement
        self.history = self.model.fit(
            X_train, y_train,
            batch_size=self.config['batch_size'],
            epochs=self.config['epochs'],
            validation_data=(X_test, y_test),
            callbacks=callbacks,
            verbose=1
        )

        # Évaluation
        loss, accuracy = self.model.evaluate(X_test, y_test)
        print(f"   ✓ Loss: {loss:.4f}")
        print(f"   ✓ Accuracy: {accuracy:.4f}")

        # Prédictions
        y_pred = np.argmax(self.model.predict(X_test), axis=1)

        # Rapport de classification
        print("\n   📊 Rapport de classification:")
        print(classification_report(y_test, y_pred))

        # Matrice de confusion
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=['Faible', 'Modéré', 'Important', 'Critique'],
                   yticklabels=['Faible', 'Modéré', 'Important', 'Critique'])
        plt.title('Matrice de confusion - Deep Learning')
        plt.ylabel('Valeur réelle')
        plt.xlabel('Valeur prédite')
        plt.tight_layout()
        plt.savefig('models/confusion_matrix_deep_learning.png')
        plt.close()

        # Courbe d'apprentissage
        plt.figure(figsize=(12, 5))

        plt.subplot(1, 2, 1)
        plt.plot(self.history.history['accuracy'])
        plt.plot(self.history.history['val_accuracy'])
        plt.title('Précision du modèle')
        plt.ylabel('Précision')
        plt.xlabel('Époque')
        plt.legend(['Train', 'Validation'], loc='lower right')

        plt.subplot(1, 2, 2)
        plt.plot(self.history.history['loss'])
        plt.plot(self.history.history['val_loss'])
        plt.title('Perte du modèle')
        plt.ylabel('Perte')
        plt.xlabel('Époque')
        plt.legend(['Train', 'Validation'], loc='upper right')

        plt.tight_layout()
        plt.savefig('models/learning_curve_deep_learning.png')
        plt.close()

        # Sauvegarde du modèle
        self.model.save('models/deep_learning_model')
        print("   ✓ Modèle de deep learning enregistré")

        return self.history

#SECTION 5: ANALYSE DE SITES WEB EN TEMPS RÉEL


In [ ]:
import os
import numpy as np
import requests
from bs4 import BeautifulSoup
import socket
import ssl
import re
from datetime import datetime
import json
import warnings
import random  # Pour simuler des résultats aléatoires

# Ignorer les avertissements liés à l'absence de vérification SSL pour les requêtes
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

class WebsiteAnalyzer:
    def __init__(self):
        # Dans cette version de démonstration, nous utilisons des modèles simulés
        self.risk_model = self.DummyModel()
        self.anomaly_model = self.DummyAnomalyModel()
        self.scaler = self.DummyScaler()

    # Classes internes pour simuler les modèles de ML
    class DummyModel:
        def predict(self, X):
            # Simuler un niveau de risque (0=Faible, 1=Modéré, 2=Important, 3=Critique)
            return np.array([random.randint(0, 3)])

        def predict_proba(self, X):
            # Simuler des probabilités pour chaque niveau
            probs = np.random.random(4)
            return np.array([probs / probs.sum()])

    class DummyScaler:
        def transform(self, X):
            # Ne fait rien, retourne simplement les données
            return X

    class DummyAnomalyModel:
        def decision_function(self, X):
            # Simuler un score d'anomalie entre -0.5 et 0.5
            return np.array([random.uniform(-0.5, 0.5)])

        def predict(self, X):
            # Simuler une détection d'anomalie (1=normal, -1=anomalie)
            return np.array([random.choice([1, -1])])

    def extract_website_features(self, url):
        """Extrait les caractéristiques d'un site web pour l'analyse"""
        # Normalisation de l'URL
        if not url.startswith(('http://', 'https://')):
            url = 'https://' + url

        domain = url.split('/')[2] if '//' in url else url.split('/')[0]
        print(f"\n🌐 Analyse du site web: {url}")

        # Caractéristiques par défaut (cas où l'analyse échoue)
        features = {
            'has_windows': 0,
            'has_linux': 0,
            'has_macos': 0,
            'has_android': 0,
            'has_ios': 0,
            'has_wordpress': 0,
            'has_joomla': 0,
            'has_drupal': 0,
            'has_oracle': 0,
            'has_microsoft': 0,
            'has_adobe': 0,
            'has_plugin': 0,
            'has_server': 1,  # Par défaut, un site web a un serveur
            'has_web': 1,     # Par défaut, c'est un site web
            'has_mobile': 0,
            'has_iot': 0,
            'has_cloud': 0,
            'has_injection': 0,
            'has_xss': 0,
            'has_sql': 0,
            'has_authentication': 0,
            'has_authorization': 0,
            'has_privilege': 0,
            'has_code_execution': 0,
            'has_arbitrary_code': 0,
            'has_buffer': 0,
            'has_overflow': 0,
            'has_dos': 0,
            'has_csrf': 0,
            'has_path_traversal': 0,
            'has_file_inclusion': 0,
            'has_encryption': 0,
            'has_ssl': 0,
            'has_mitm': 0,
            'has_bypass': 0,
            'has_backdoor': 0,
            'has_zero_day': 0,
            'risk_level': 0,
            'impact_level': 0
        }

        detected_vulnerabilities = []
        security_recommendations = []

        try:
            # Préparation de l'en-tête des requêtes
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }

            # Vérification SSL
            print("   - Vérification du certificat SSL...")
            try:
                context = ssl.create_default_context()
                with socket.create_connection((domain, 443), timeout=10) as sock:
                    with context.wrap_socket(sock, server_hostname=domain) as ssock:
                        cert = ssock.getpeercert()

                        # Le certificat existe
                        features['has_ssl'] = 1

                        # Vérification de la date d'expiration
                        expiry_date = datetime.strptime(cert['notAfter'], '%b %d %H:%M:%S %Y %Z')
                        ssl_expiry = expiry_date < datetime.now()
                        if ssl_expiry:
                            detected_vulnerabilities.append({
                                'name': 'Certificat SSL expiré',
                                'description': f"Le certificat SSL a expiré le {expiry_date.strftime('%Y-%m-%d')}",
                                'severity': 'Important'
                            })
                            security_recommendations.append({
                                'title': 'Renouveler le certificat SSL',
                                'description': 'Contactez votre fournisseur de certificat ou utilisez Let\'s Encrypt pour renouveler votre certificat SSL.',
                                'priority': 'Haute'
                            })
                        print(f"      ✓ Certificat SSL valide: {'Non' if ssl_expiry else 'Oui'}")
            except Exception as e:
                features['has_ssl'] = 0
                detected_vulnerabilities.append({
                    'name': 'Absence de SSL/TLS',
                    'description': 'Le site n\'utilise pas de connexion sécurisée (HTTPS) ou le certificat n\'est pas accessible',
                    'severity': 'Critique'
                })
                security_recommendations.append({
                    'title': 'Mettre en place HTTPS',
                    'description': 'Installez un certificat SSL via Let\'s Encrypt ou un autre fournisseur.',
                    'priority': 'Critique'
                })
                print(f"      ⚠️ Pas de certificat SSL ou erreur: {e}")

            # Récupération de la page
            print("   - Récupération du contenu de la page...")
            response = requests.get(url, headers=headers, timeout=10, verify=False)
            soup = BeautifulSoup(response.content, 'html.parser')

            # Simulation de détection de vulnérabilités
            # Pour la démonstration, nous ajoutons quelques vulnérabilités aléatoires
            vuln_types = [
                ('has_xss', 'Vulnérabilité XSS potentielle', 'Scripts externes suspects détectés'),
                ('has_authentication', 'Formulaire d\'identification non sécurisé', 'Le formulaire d\'authentification pourrait être vulnérable'),
                ('has_sql', 'Injection SQL potentielle', 'Paramètres URL non sanitisés détectés'),
                ('has_csrf', 'Protection CSRF manquante', 'Manque de jetons CSRF sur les formulaires'),
                ('has_path_traversal', 'Traversée de chemin possible', 'Protection insuffisante contre les attaques de traversée de chemin')
            ]

            # Simuler aléatoirement 1 à 3 vulnérabilités
            num_vulnerabilities = random.randint(1, 3)
            selected_vulnerabilities = random.sample(vuln_types, num_vulnerabilities)

            severities = ['Faible', 'Modéré', 'Important', 'Critique']
            priorities = ['Basse', 'Moyenne', 'Haute', 'Critique']

            for vuln_type, name, desc in selected_vulnerabilities:
                features[vuln_type] = 1
                severity = random.choice(severities)
                priority = random.choice(priorities)

                detected_vulnerabilities.append({
                    'name': name,
                    'description': desc,
                    'severity': severity
                })

                security_recommendations.append({
                    'title': f'Corriger {name.lower()}',
                    'description': f'Mettre en place des mesures pour protéger contre {name.lower()}',
                    'priority': priority
                })

        except Exception as e:
            print(f"⚠️ Erreur lors de l'analyse du site: {e}")
            # En cas d'erreur, ajouter une vulnérabilité générique
            detected_vulnerabilities.append({
                'name': 'Erreur lors de l\'analyse',
                'description': f'Impossible d\'analyser correctement le site: {str(e)}',
                'severity': 'Inconnu'
            })
            security_recommendations.append({
                'title': 'Vérifier l\'accessibilité du site',
                'description': 'Assurez-vous que le site est accessible et répond correctement aux requêtes HTTP.',
                'priority': 'Haute'
            })

        return features, detected_vulnerabilities, security_recommendations

    def analyze_website(self, url):
        """Analyse complète d'un site web et prédiction des risques"""
        try:
            # Extraction des caractéristiques
            features, detected_vulnerabilities, security_recommendations = self.extract_website_features(url)

            # Conversion des caractéristiques en format attendu par le modèle
            X = np.array([[features[key] for key in features]])

            print(f"Nombre de caractéristiques extraites : {X.shape[1]}")  # Débogage

            # Normalisation
            X_scaled = self.scaler.transform(X)

            # Prédiction du niveau de risque
            print("\n🔮 Prédiction des risques de sécurité...")
            risk_level = self.risk_model.predict(X_scaled)[0]

            # Probabilités pour chaque classe
            risk_proba = self.risk_model.predict_proba(X_scaled)[0]

            # Détection d'anomalies
            anomaly_score = self.anomaly_model.decision_function(X_scaled)[0]
            is_anomaly = self.anomaly_model.predict(X_scaled)[0] == -1

            # Mappage du niveau de risque numérique au texte
            risk_level_map = {
                0: 'Faible',
                1: 'Modéré',
                2: 'Important',
                3: 'Critique'
            }

            # Calcul du score global (0-100, où 100 = risque élevé)
            risk_score_pct = int(((risk_level / 3) * 60) +
                             (len(detected_vulnerabilities) * 3) +
                             (20 if is_anomaly else 0))

            # Plafonnement à 100
            risk_score_pct = min(100, risk_score_pct)

            # Résultat
            result = {
                'url': url,
                'scan_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                'risk_level_numeric': int(risk_level),
                'risk_level_text': risk_level_map.get(int(risk_level), 'Inconnu'),
                'risk_score': risk_score_pct,
                'is_anomaly': bool(is_anomaly),
                'anomaly_score': float(anomaly_score),
                'vulnerability_count': len(detected_vulnerabilities),
                'vulnerabilities': detected_vulnerabilities,
                'recommendations': security_recommendations,
                'probability': {
                    'Faible': float(risk_proba[0]) if len(risk_proba) > 0 else 0,
                    'Modéré': float(risk_proba[1]) if len(risk_proba) > 1 else 0,
                    'Important': float(risk_proba[2]) if len(risk_proba) > 2 else 0,
                    'Critique': float(risk_proba[3]) if len(risk_proba) > 3 else 0
                }
            }

            # Affichage du résultat
            print(f"\n📊 Résultats de l'analyse:")
            print(f"   ✓ URL: {result['url']}")
            print(f"   ✓ Niveau de risque: {result['risk_level_text']} ({result['risk_score']}%)")
            print(f"   ✓ Anomalie détectée: {'Oui' if result['is_anomaly'] else 'Non'}")
            print(f"   ✓ Nombre de vulnérabilités: {result['vulnerability_count']}")
            print("   ✓ Probabilités par niveau de risque:")
            for level, prob in result['probability'].items():
                print(f"      - {level}: {prob:.2f}")

            return result

        except Exception as e:
            return {
                'error': f'Erreur lors de l\'analyse: {str(e)}',
                'url': url,
                'scan_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }

def generate_security_report(result, output_format='json'):
    """
    Génère un rapport de sécurité au format JSON ou HTML

    Args:
        result: Dictionnaire contenant les résultats de l'analyse
        output_format: Format du rapport ('json' ou 'html')

    Returns:
        str: Nom du fichier de rapport généré
    """
    # Vérification que le résultat est valide
    if not result or 'error' in result:
        error_message = result.get('error', 'Erreur inconnue') if result else 'Résultat vide'
        print(f"\n⚠️ Impossible de générer le rapport complet: {error_message}")

        # Créer un rapport d'erreur minimal
        result = {
            'url': result.get('url', 'URL inconnue') if result else 'URL inconnue',
            'scan_date': result.get('scan_date', datetime.now().strftime("%Y-%m-%d %H:%M:%S")) if result else datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'error': error_message,
            'risk_level_text': 'Inconnu',
            'risk_score': 0,
            'vulnerability_count': 0,
            'vulnerabilities': [],
            'recommendations': []
        }

    # S'assurer que toutes les clés nécessaires existent
    required_keys = ['url', 'scan_date', 'risk_level_text', 'risk_score', 'vulnerability_count',
                   'vulnerabilities', 'recommendations']

    for key in required_keys:
        if key not in result:
            result[key] = 'Non disponible' if key in ['url', 'scan_date', 'risk_level_text'] else 0 if key in ['risk_score', 'vulnerability_count'] else []

    # Création du répertoire reports s'il n'existe pas
    os.makedirs('reports', exist_ok=True)

    # Création du nom de fichier en fonction de l'URL
    domain = result['url'].replace('https://', '').replace('http://', '').split('/')[0]
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    print(f"\n📝 Génération du rapport de sécurité...")

    if output_format.lower() == 'json':
        # Génération du rapport JSON
        filename = f"reports/rapport_securite_{domain}_{timestamp}.json"
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(result, f, ensure_ascii=False, indent=4)

    elif output_format.lower() == 'html':
        # Génération du rapport HTML avec un style moderne
        filename = f"reports/rapport_securite_{domain}_{timestamp}.html"

        # Préparation des sections HTML pour les vulnérabilités
        vulns_html = ""
        if result['vulnerabilities']:
            for vuln in result['vulnerabilities']:
                severity = vuln.get('severity', 'Modéré')
                description = vuln.get('description', 'Aucune description disponible')
                vulns_html += f"""
                <div class="vuln-item">
                    <h4>{vuln['name']} <span class="severity severity-{severity}">{severity}</span></h4>
                    <p>{description}</p>
                </div>
                """
        else:
            vulns_html = "<p>Aucune vulnérabilité détectée.</p>"

        # Préparation des sections HTML pour les recommandations
        recs_html = ""
        if result['recommendations']:
            for rec in result['recommendations']:
                priority = rec.get('priority', 'Moyenne')
                description = rec.get('description', 'Aucune description disponible')
                recs_html += f"""
                <div class="rec-item">
                    <h4>{rec['title']}</h4>
                    <p><span class="priority priority-{priority}">Priorité: {priority}</span></p>
                    <p>{description}</p>
                </div>
                """
        else:
            recs_html = "<p>Aucune recommandation disponible.</p>"

        # Déterminer la classe CSS pour le niveau de risque
        risk_class = 'risk-low'
        if result['risk_level_text'] == 'Critique':
            risk_class = 'risk-critical'
        elif result['risk_level_text'] == 'Important':
            risk_class = 'risk-important'
        elif result['risk_level_text'] == 'Modéré':
            risk_class = 'risk-moderate'

        html_content = f"""<!DOCTYPE html>
<html lang="fr">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Rapport de sécurité - {result['url']}</title>
    <style>
        :root {{
            --color-bg: #f5f8fa;
            --color-text: #333;
            --color-primary: #2563eb;
            --color-secondary: #4b5563;
            --color-accent: #60a5fa;
            --color-success: #10b981;
            --color-warning: #f59e0b;
            --color-danger: #ef4444;
            --color-info: #3b82f6;
            --color-light: #f3f4f6;
            --color-dark: #1f2937;
            --color-white: #ffffff;
            --shadow-sm: 0 1px 2px 0 rgba(0, 0, 0, 0.05);
            --shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1), 0 2px 4px -1px rgba(0, 0, 0, 0.06);
            --shadow-lg: 0 10px 15px -3px rgba(0, 0, 0, 0.1), 0 4px 6px -2px rgba(0, 0, 0, 0.05);
            --rounded: 0.375rem;
        }}

        body {{
            font-family: 'Segoe UI', system-ui, -apple-system, sans-serif;
            line-height: 1.6;
            color: var(--color-text);
            background-color: var(--color-bg);
            margin: 0;
            padding: 0;
        }}

        .container {{
            max-width: 1200px;
            margin: 2rem auto;
            padding: 0 1rem;
        }}

        header {{
            background-color: var(--color-primary);
            color: var(--color-white);
            padding: 2rem 0;
            margin-bottom: 2rem;
            box-shadow: var(--shadow);
        }}

        header .container {{
            display: flex;
            justify-content: space-between;
            align-items: center;
            margin-top: 0;
            margin-bottom: 0;
        }}

        h1, h2, h3, h4 {{
            color: var(--color-dark);
            margin-top: 1.5rem;
            margin-bottom: 1rem;
        }}

        header h1 {{
            color: var(--color-white);
            margin: 0;
            font-size: 1.8rem;
        }}

        .card {{
            background-color: var(--color-white);
            border-radius: var(--rounded);
            padding: 1.5rem;
            margin-bottom: 1.5rem;
            box-shadow: var(--shadow);
        }}

        .summary {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
            gap: 1rem;
            margin-bottom: 2rem;
        }}

        .summary-item {{
            padding: 1rem;
            border-radius: var(--rounded);
            background-color: var(--color-white);
            box-shadow: var(--shadow-sm);
            text-align: center;
        }}

        .summary-item h3 {{
            margin-top: 0;
            color: var(--color-secondary);
            font-size: 1rem;
        }}

        .summary-item p {{
            font-size: 1.5rem;
            font-weight: bold;
            margin: 0.5rem 0;
        }}

        .vuln-item, .rec-item {{
            border-left: 4px solid var(--color-secondary);
            margin-bottom: 1rem;
            padding: 1rem;
            background-color: var(--color-light);
            border-radius: 0 var(--rounded) var(--rounded) 0;
        }}

        .vuln-item h4, .rec-item h4 {{
            margin-top: 0;
            margin-bottom: 0.5rem;
        }}

        .severity {{
            display: inline-block;
            padding: 0.25rem 0.75rem;
            border-radius: 2rem;
            font-size: 0.75rem;
            font-weight: bold;
            margin-left: 0.5rem;
            color: var(--color-white);
        }}

        .severity-Critique {{
            background-color: var(--color-danger);
        }}

        .severity-Important {{
            background-color: var(--color-warning);
        }}

        .severity-Modéré {{
            background-color: var(--color-info);
        }}

        .severity-Faible {{
            background-color: var(--color-success);
        }}

        .risk-score {{
            font-size: 1.2rem;
            font-weight: bold;
            color: var(--color-white);
            padding: 0.5rem 1rem;
            border-radius: var(--rounded);
            display: inline-block;
        }}

        .risk-low {{
            background-color: var(--color-success);
        }}

        .risk-moderate {{
            background-color: var(--color-info);
        }}

        .risk-important {{
            background-color: var(--color-warning);
        }}

        .risk-critical {{
            background-color: var(--color-danger);
        }}

        .priority {{
            font-weight: bold;
        }}

        .priority-Critique, .priority-Haute {{
            color: var(--color-danger);
        }}

        .priority-Moyenne {{
            color: var(--color-warning);
        }}

        .priority-Basse {{
            color: var(--color-info);
        }}

        footer {{
            text-align: center;
            margin-top: 2rem;
            padding: 1rem 0;
            color: var(--color-secondary);
            font-size: 0.875rem;
        }}

        @media (max-width: 768px) {{
            .summary {{
                grid-template-columns: 1fr;
            }}
        }}
    </style>
</head>
<body>
    <header>
        <div class="container">
            <h1>Rapport de sécurité web</h1>
            <p>{result['scan_date']}</p>
        </div>
    </header>

    <div class="container">
        <div class="card">
            <h2>Informations générales</h2>
            <p><strong>URL analysée:</strong> {result['url']}</p>
            <p><strong>Date d'analyse:</strong> {result['scan_date']}</p>
            <p>
                <strong>Niveau de risque:</strong>
                <span class="risk-score {risk_class}">
                    {result['risk_level_text']} ({result['risk_score']}%)
                </span>
            </p>
        </div>

        <div class="summary">
            <div class="summary-item">
                <h3>Niveau de risque</h3>
                <p>{result['risk_level_text']}</p>
            </div>
            <div class="summary-item">
                <h3>Score de risque</h3>
                <p>{result['risk_score']}%</p>
            </div>
            <div class="summary-item">
                <h3>Vulnérabilités détectées</h3>
                <p>{result['vulnerability_count']}</p>
            </div>
        </div>

        <div class="card">
            <h2>Vulnérabilités détectées</h2>
            {vulns_html}
        </div>

        <div class="card">
            <h2>Recommandations de sécurité</h2>
            {recs_html}
        </div>
    </div>

    <footer>
        <div class="container">
            <p>Rapport généré automatiquement le {result['scan_date']}</p>
            <p>© {datetime.now().year} Système d'analyse de vulnérabilités web</p>
        </div>
    </footer>
</body>
</html>
"""

        with open(filename, 'w', encoding='utf-8') as f:
            f.write(html_content)

    else:
        print(f"\n⚠️ Format de rapport non supporté: {output_format}")
        return None

    print(f"\n✅ Rapport généré avec succès: {filename}")
    return filename




def evaluate_website(url):
    """Fonction qui appelle l'analyseur de site web"""
    try:
        # Création de l'objet analyseur
        analyzer = WebsiteAnalyzer()

        # Analyse du site web
        result = analyzer.analyze_website(url)

        return result
    except Exception as e:
        return {
            'error': f'Erreur lors de l\'évaluation: {str(e)}',
            'url': url,
            'scan_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

def main():
    """Fonction principale du programme"""
    print("\n" + "="*80)
    print("🚀 DÉMARRAGE DU SYSTÈME DE PRÉDICTION DE VULNÉRABILITÉS WEB")
    print("="*80)

    # Mode d'exécution (entraînement ou analyse)
    mode = input("\nMode d'exécution :\n1. Entraînement des modèles\n2. Analyse d'un site web\nChoix (1/2): ").strip()

    if mode == '1':
        # Mode entraînement (simulé pour cet exemple)
        print("\n🔄 Mode: Entraînement des modèles")
        print("\n❗ La fonctionnalité d'entraînement est désactivée dans cette version de démonstration.")

    elif mode == '2':
        # Mode analyse
        print("\n🔍 Mode: Analyse d'un site web")

        # Demande de l'URL à analyser
        url = input("\nEntrez l'URL du site à analyser: ").strip()

        # Analyse du site
        result = evaluate_website(url)

        # Génération du rapport
        report_format = input("\nFormat du rapport (json/html): ").strip().lower()
        if report_format not in ['json', 'html']:
            report_format = 'json'

        report_file = generate_security_report(result, report_format)

    else:
        print("\n⚠️ Mode non reconnu. Fin du programme.")

    print("\n✅ Programme terminé avec succès!")

# Si ce fichier est exécuté directement
if __name__ == "__main__":
    main()


🚀 DÉMARRAGE DU SYSTÈME DE PRÉDICTION DE VULNÉRABILITÉS WEB

Mode d'exécution :
1. Entraînement des modèles
2. Analyse d'un site web
Choix (1/2): 2

🔍 Mode: Analyse d'un site web

Entrez l'URL du site à analyser: https://www.fbi.gov/services

🌐 Analyse du site web: https://www.fbi.gov/services
   - Vérification du certificat SSL...
      ✓ Certificat SSL valide: Oui
   - Récupération du contenu de la page...
Nombre de caractéristiques extraites : 39

🔮 Prédiction des risques de sécurité...

📊 Résultats de l'analyse:
   ✓ URL: https://www.fbi.gov/services
   ✓ Niveau de risque: Important (46%)
   ✓ Anomalie détectée: Non
   ✓ Nombre de vulnérabilités: 2
   ✓ Probabilités par niveau de risque:
      - Faible: 0.08
      - Modéré: 0.39
      - Important: 0.26
      - Critique: 0.26

Format du rapport (json/html): html

📝 Génération du rapport de sécurité...

✅ Rapport généré avec succès: reports/rapport_securite_www.fbi.gov_20250506_220937.html

✅ Programme terminé avec succès!


In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import os # Add this line at the beginning of your script to import the os module

# Fonction principale
if __name__ == "__main__":
    # Chemin vers le fichier de données
    data_file = "/content/rapports_dgssi_securite_cleaned.csv"

    # Configuration personnalisée (optionnelle)
    custom_config = {
        'preprocessing': {
            'feature_selection': True,
            'n_features': 25,
            'apply_smote': True
        },
        'models': {
            'xgboost': {
                'grid_search': True
            }
        }
    }

    # Exemple d'utilisation - Entraînement complet
    print("\n" + "="*80)
    print("🚀 DÉMARRAGE DU SYSTÈME DE PRÉDICTION DE VULNÉRABILITÉS WEB")
    print("="*80)

    # Mode d'exécution (entraînement ou analyse)
    mode = input("\nMode d'exécution :\n1. Entraînement des modèles\n2. Analyse d'un site web\nChoix (1/2): ").strip()

    if mode == '1':
        # Mode entraînement
        print("\n🔄 Mode: Entraînement des modèles")

        # Vérification de l'existence du fichier de données
        if not os.path.exists(data_file):
            data_file = input(f"\n⚠️ Fichier {data_file} non trouvé. Veuillez entrer le chemin du fichier de données: ").strip()

        # Entraînement du pipeline complet
        pipeline_results = train_complete_pipeline(data_file, custom_config)

        print(f"\n🏆 Meilleur modèle: {pipeline_results['best_model_name']} (Score: {pipeline_results['best_score']:.4f})")

    elif mode == '2':
        # Mode analyse
        print("\n🔍 Mode: Analyse d'un site web")

        # Demande de l'URL à analyser
        url = input("\nEntrez l'URL du site à analyser: ").strip()

        # Analyse du site
        result = evaluate_website(url)

        # Génération du rapport
        report_format = input("\nFormat du rapport (json/html): ").strip().lower()
        if report_format not in ['json', 'html']:
            report_format = 'json'

        report_file = generate_security_report(result, report_format)

    else:
        print("\n⚠️ Mode non reconnu. Fin du programme.")

    print("\n✅ Programme terminé avec succès!")


🚀 DÉMARRAGE DU SYSTÈME DE PRÉDICTION DE VULNÉRABILITÉS WEB

Mode d'exécution :
1. Entraînement des modèles
2. Analyse d'un site web
Choix (1/2): 1

🔄 Mode: Entraînement des modèles

📂 Création de la structure de répertoires...
   ✓ Structure de répertoires créée

📂 Chargement des données depuis: /content/rapports_dgssi_securite_cleaned.csv...
   ✓ Données chargées: 1517 entrées, 9 colonnes
   - Prétraitement des données...
   ✓ Données prétraitées.

🔧 Extraction des caractéristiques...
   ⚠️ Valeurs manquantes détectées dans Risk Level Numeric. Remplacement par 0...
   - Utilisation de la colonne 'Combined_Text' pour l'extraction textuelle
   ✓ Caractéristiques textuelles extraites: 1000 dimensions
   - Extraction des caractéristiques basées sur les règles...


   🔄 Traitement des entrées: 100%|██████████| 1517/1517 [00:00<00:00, 7354.21it/s]


   ✓ Caractéristiques basées sur les règles extraites: 39 dimensions
   ⚠️ Valeurs manquantes détectées dans les caractéristiques. Remplacement par 0...
   ✓ Scaling appliqué: standard
   - Sélection des 25 meilleures caractéristiques...
   ✓ Sélection de caractéristiques appliquée: 25 dimensions retenues
   📋 Caractéristiques sélectionnées: ['has_windows', 'has_linux', 'has_macos', 'has_ios', 'has_wordpress', 'has_joomla', 'has_drupal', 'has_adobe', 'has_plugin', 'has_server', 'has_web', 'has_cloud', 'has_xss', 'has_sql', 'has_authentication', 'has_authorization', 'has_privilege', 'has_code_execution', 'has_arbitrary_code', 'has_overflow', 'has_dos', 'has_path_traversal', 'has_bypass', 'has_zero_day', 'impact_level']

⚖️ Application de techniques d'équilibrage des classes...
   - Distribution initiale des classes: {0.0: 14, 1.0: 218, 2.0: 817, 3.0: 468}
   ✓ Distribution après SMOTE: {0.0: 817, 1.0: 817, 2.0: 817, 3.0: 817}

🤖 Entraînement et évaluation des modèles...
   - Données div